## 크롤링 crawling

##### 홈페이지 앞에 view-source:를 붙이면 홈페이지의 소스 확인 가능
##### 일반화:
##### <여는태그>내용</닫는태그>
##### <여는태그 속성명="속성값">내용</닫는태그>
##### <여는태그 속성명="속성값"/>
##### html부터 /html까지 하나의 덩어리, head부터 /head까지 하나의 덩어리, body부터 /body까지 하나의 덩어리
##### l는 end를 의미함

In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import csv

##### import A는 A를 메모리로 불러옴
##### 상위폴더 안에 있는 하위폴더를 한번에 불러오고 싶으면 from 상위폴더 import 하위폴더의 형태로 쓰면 됨

In [ ]:
html = urlopen("http://www.naver.com")
html

##### urlopen을 통해 받아온 네이버를 BeautifulSoup을 통해 실행하면 view-sorce:https://naver.com의 내용을 받아올 수 있음

## 웹문서 전체를 가져오기

In [ ]:
bsObject = BeautifulSoup(html, "html.parser")
bsObject.html

##### bsObject.html은 html 부분의 내용을 받음. 이 속에서 head, title 부분을 가져올 수도 있음

## head 가져오기

In [ ]:
bsObject.head

##### head 부분에서 8번째 줄은 /가 없기 때문에 한줄로 끝나지 않음. 맨 마지막 줄의 마지막 /meta에서 끝이 남

## title 가져오기

In [ ]:
bsObject.head.title

##### head 부분에서 title 가져오기

In [ ]:
bsObject.head.title.get_text()  # bsObject.head.title.text.strip()

##### title 부분에서 꺽쇠부분을 제외한 나머지를 가져오기

## meta 데이터 가져오기

In [ ]:
bsObject.head.find_all('meta')

##### find_all 함수를 통해 meta 태그를 찾아서, 정보를 제일 큰 범위의 meta부터 차례로 중복해서 가져오기

## meta 데이터의 모든 content 가져오기

In [ ]:
for meta in bsObject.head.find_all('meta'):
    print(meta.get('content'))

##### meta 태그 내에 있는 content property를 가져오기. 첫번째 meta에 content가 없다면 None을 출력
##### 궁금한 점 : 왜 content는 출력할 때 중복되지 않을까?

## meta 데이터 중 원하는 tag 가져오기

In [ ]:
bsObject.head.find("meta", {"name":"description"})

##### 모든 meta에서 name과 description이 있는 부분을 통으로 다 가져오는 것

In [ ]:
bsObject.head.find("meta", {"name":"description"}).get('content')

##### name과 description이 있는 줄에서 content 부분만 가져오는 것. 이때는 통으로 가져오는 것이 아니라 그 줄에서만 가져오는 것.

In [ ]:
bsObject.head.find("meta", {"content":"네이버"}).get('property')

##### content가 네이버인 그 줄에서 property만 가져오는 것

## 모든 링크의 텍스트와 주소 가져오기

In [ ]:
for link in bsObject.find_all('a'):
    print(link.get_text(), link.get('href'))

##### 첫번째 'a' 부분의 텍스트만 받아서 출력하고, 'href'라는 부분의 value값을 받아 옴 (<a를 찾아보기)

## 예제: 네이버 베스트셀러 주간 순위에 있는 책의 정보를 가져오기
첫페이지에 있는 25권의 책목록에서 링크를 추출하여 각 추출된 링크로 접속하여 책이름, 저자, 가격 정보 등을 출력

In [ ]:
html = urlopen('https://book.naver.com/bestsell/bestseller_list.nhn')
bsObject = BeautifulSoup(html, "html.parser")

In [ ]:
bsObject

##### 인터넷에서 crawling하고 싶은 링크의 소스를 가져온 것

In [ ]:
for n in range(0,25):
    dt = bsObject.find("dt", {"id":"book_title_"+str(n)})
    title = dt.select('a')[0].get_text()
    author = dt.next_sibling.next_sibling.select('a')[0].get_text()
    publisher = dt.next_sibling.next_sibling.get_text().split()[-1]
    print(n+1, title, author, publisher)

##### book_title_0, book_title_1 이렇게 표현되는 것이 clue -> dt tag에 들어가서 book_title_숫자면 title을 가져오게 코드를 짜기
##### Title : 'a' tag를 찾아서 첫번쨰 것의 범위를 설정하고, 그 범위 안에서 텍스트를 가져오라는 뜻
##### Author : next sibling은 바로 그 다음에 나오는 tag를 찾으라는 것. Enter도 하나의 tag로 간주하기 때문에 두번 사용하여 dd tag가 설정되고, select와 get을 통해 a부분의 text를 찾음
##### Publisher : dd 전체에서 get.text를 하면 여러 텍스트가 남는데, 이 상태에서 split을 쓰면 여러 텍스트를 list로 만들고, 거기에서 제일 마지막의 것을 쓰기 위해 -1을 붙임

In [ ]:
# Create a file to write to, add headers row
with open('crawling.csv', 'w') as f:
    writer = csv.writer(f)
    writer.writerow(['rank', 'title', 'author', 'publisher'])
        
    for n in range(0,25):
        dt = bsObject.find("dt", {"id":"book_title_"+str(n)})
        title = dt.select('a')[0].get_text()
        author = dt.next_sibling.next_sibling.select('a')[0].get_text()
        publisher = dt.next_sibling.next_sibling.get_text().split()[-1]
        writer.writerow([n+1, title, author, publisher])

##### 엑셀파일 만들기
##### crawling한 것을 file로 열고, writer.writerow를 통해 header처럼 각각의 줄에 제목을 적어줌. for, dt, title, author, publisher 코드 부분은 위와 동일하게 적고 마지막에 writer.writerow 부분을 적고 실행하면 텍스트파일이 저장됨.
##### 텍스트 파일을 엑셀로 불러오면 한글이 깨지기 때문에, 새 엑셀 파일에 'Import>CSV파일>Delimited>comma'를 하면 잘 됨.